In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from langchain.schema.runnable import RunnableBranch,RunnableLambda

In [15]:
load_dotenv()

True

In [16]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7)
parser = StrOutputParser()

In [17]:
class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser_two = PydanticOutputParser(pydantic_object=Feedback)


In [18]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser_two.get_format_instructions()}
)

In [19]:
classify_chain = prompt1 | model | parser_two

In [20]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [21]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [22]:
chain = classify_chain | branch_chain

In [23]:
result = chain.invoke(
    {
        "feedback" : "This is a great smartphones"
    }
)

In [24]:
print(result)

Several responses are appropriate, depending on the context. Here are a few options:

**Formal:**

* "Thank you for your kind words. We appreciate your positive feedback."
* "We're delighted to hear you had a positive experience.  We strive to provide the best possible service/product."
* "Thank you for taking the time to share your feedback.  We're pleased to know you're satisfied."


**Informal:**

* "Thanks so much! We really appreciate it."
* "Glad to hear you liked it!"
* "Awesome, thanks for letting us know!"


**If you want to encourage more feedback:**

* "Thank you for your positive feedback! We're always looking for ways to improve, so if you have any further suggestions, please don't hesitate to share them."
* "We're so happy you had a great experience.  We'd love to hear more about what you enjoyed."


**If the feedback was specific to something:**

* "Thank you! We're especially proud of [the specific thing they mentioned] and we're glad you enjoyed it."


Choose the respo

In [25]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
 +----------------------+  
 | PydanticOutputParser |  
 +----------------------+  
             *             
             *             
             *             
        +--------+         
        | Branch |         
        +--------+         
             *             
             *             
             *             
     +--------------+      
     | BranchOutput |      
     +--------------+      
